In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras

c:\Users\Raaghav\miniconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
train = tf.keras.utils.image_dataset_from_directory(
    directory = 'New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
    labels = 'inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(128, 128),
    seed=None,
    shuffle=True,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
    )



Found 70295 files belonging to 38 classes.


In [4]:
valid = tf.keras.utils.image_dataset_from_directory(
    directory='New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
    labels='inferred',
    label_mode='categorical',
    seed=None,
    batch_size=32,
    image_size=(128, 128),   
    shuffle=True,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False  
)

Found 17572 files belonging to 38 classes.


In [5]:
class_names = train.class_names
print("Classes found:", class_names)
num_classes = len(class_names)
print("Number of classes:", num_classes)


Classes found: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tom

In [6]:
train = train.apply(tf.data.experimental.ignore_errors())


In [7]:
valid = valid.apply(tf.data.experimental.ignore_errors())


In [8]:
def preprocess(images, labels):
    return images, labels

train = train.map(preprocess)
valid = valid.map(preprocess)


In [9]:
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.layers import RandomFlip,RandomRotation


In [10]:
from keras import mixed_precision
from keras import Input
from keras.layers import Dropout
mixed_precision.set_global_policy('mixed_float16')

model = Sequential()


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 5060 Laptop GPU, compute capability 12.0


In [11]:
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [12]:

model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [13]:

model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [14]:

model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [15]:

'''model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))'''

"model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))\nmodel.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,activation='relu'))\nmodel.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))"

In [16]:
model.add(tf.keras.layers.Dropout(0.3))

In [17]:
model.add(Flatten())

In [18]:
model.add(tf.keras.layers.Dense(units=1000,activation='relu'))

In [19]:
model.add(tf.keras.layers.Dropout(0.4)) 

In [20]:
model.add(tf.keras.layers.Dense(units=38,activation='softmax'))

In [21]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])


In [22]:
model.fit(train, batch_size=32,epochs=5, validation_data= valid ,callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(patience=3),
    ],)

Epoch 1/5
2197/2197 [==============================] - 376s 63ms/step - loss: 1.2756 - accuracy: 0.6339 - val_loss: 0.4819 - val_accuracy: 0.8534 - lr: 1.0000e-04
Epoch 2/5
2197/2197 [==============================] - 142s 65ms/step - loss: 0.4157 - accuracy: 0.8693 - val_loss: 0.2669 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 3/5
2197/2197 [==============================] - 142s 65ms/step - loss: 0.2531 - accuracy: 0.9183 - val_loss: 0.2156 - val_accuracy: 0.9306 - lr: 1.0000e-04
Epoch 4/5
2197/2197 [==============================] - 144s 65ms/step - loss: 0.1766 - accuracy: 0.9420 - val_loss: 0.1737 - val_accuracy: 0.9464 - lr: 1.0000e-04
Epoch 5/5
2197/2197 [==============================] - 144s 65ms/step - loss: 0.1302 - accuracy: 0.9572 - val_loss: 0.1410 - val_accuracy: 0.9565 - lr: 1.0000e-04


In [23]:
model.save('final_model.keras')

In [24]:

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
    labels = 'inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(128, 128),
    seed=42,
    shuffle=False,
     validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    )

Found 70295 files belonging to 38 classes.


In [43]:
import cv2
image_path = 'grey-leaf-spot-of-maize-maize-1.JPG'
# Reading an image in default mode
img = cv2.imread(image_path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Converting BGR to RGB


In [44]:

image = tf.keras.preprocessing.image.load_img(image_path,target_size=(128,128))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = model.predict(input_arr)
result_index = np.argmax(predictions) #Return index of max element
print(result_index)

1/1 [==============================] - 0s 18ms/step
7


In [45]:
class_name = train_ds.class_names
print(class_name)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [46]:
model_prediction = class_name[result_index]
print(model_prediction)

Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
